In [17]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pickle

def load_data(i):
    with open('drive/My Drive/Machine_Learning-prj/colorization/X_'+str(i)+'.pkl', 'rb') as f:
        images = pickle.load(f)

    with open('drive/My Drive/Machine_Learning-prj/colorization/y_'+str(i)+'.pkl', 'rb') as f:
        gt_images = pickle.load(f)
    
    return images, gt_images

In [0]:
images_val, gt_images_val = load_data(10)

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import applications

In [20]:
resnet = applications.resnet50.ResNet50(weights=None, classes=365)
resnet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
x = resnet.output
model_tmp = Model(inputs = resnet.input, outputs = x) 

In [22]:
layer_3, layer_7, layer_13, layer_16 = model_tmp.get_layer('activation_9').output, model_tmp.get_layer('activation_21').output, model_tmp.get_layer('activation_39').output, model_tmp.get_layer('activation_48').output

ValueError: ignored

In [0]:
fcn1 = Conv2D(filters=2 , kernel_size=1, name='fcn1')(layer_16)

fcn2 = Conv2DTranspose(filters=layer_13.get_shape().as_list()[-1] , kernel_size=4, strides=2, padding='same', name='fcn2')(fcn1)
fcn2_skip_connected = Add(name="fcn2_plus_layer13")([fcn2, layer_13])

fcn3 = Conv2DTranspose(filters=layer_7.get_shape().as_list()[-1], kernel_size=4, strides=2, padding='same', name="fcn3")(fcn2_skip_connected)
fcn3_skip_connected = Add(name="fcn3_plus_layer_7")([fcn3, layer_7])

fcn4 = Conv2DTranspose(filters=layer_3.get_shape().as_list()[-1], kernel_size=4, strides=2, padding='same', name="fcn4")(fcn3_skip_connected)
fcn4_skip_connected = Add(name="fcn4_plus_layer_3")([fcn4, layer_3])

# Upsample again
fcn5 = Conv2DTranspose(filters=2, kernel_size=16, strides=(4, 4), padding='same', name="fcn5")(fcn4_skip_connected)
relu255 = ReLU(max_value=255) (fcn5)

In [0]:
model = Model(inputs = resnet.input, outputs = relu255)
model.summary()

In [0]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
    

model.compile(loss=root_mean_squared_error, optimizer='rmsprop')

In [0]:
from random import randint
import cv2

def preprocessing_input_output(img, gt):
    img_n = img.copy()
    img_n = cv2.cvtColor(img_n, cv2.COLOR_GRAY2BGR)
  
    return img_n, gt

In [0]:
import numpy as np
from keras.utils import to_categorical

def image_batch_generator(images, gt_images, batch_size):
    while True:
        batch_paths = np.random.choice(a=len(images), size=batch_size)
        input = []
        output = []
        
        for i in batch_paths:
            in_p, out_p = preprocessing_input_output(images[i], gt_images[i])
            input.append(in_p)
            output.append(out_p)
        
        input = np.array(input)
        output = np.array(output)
        # output = np.array(np.expand_dims(output, -1))
        yield(input, output)

Using TensorFlow backend.


In [0]:
from keras.callbacks import ModelCheckpoint

filepath = 'drive/My Drive/Machine_Learning-prj/colorization/colorization_12_1.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, save_weights_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
model.load_weights(filepath)

In [0]:
batch_size = 32

for i in range(0, 150):
    i = np.random.randint(1, 9)
    images, gt_images = load_data(i)
    model.fit_generator(generator=image_batch_generator(images, gt_images, batch_size),
                       steps_per_epoch=len(images)//batch_size,
                       epochs=5,
                       verbose=1,
                       validation_data=image_batch_generator(images_val, gt_images_val, batch_size),
                       validation_steps=len(images_val)//batch_size,
                       callbacks=callbacks_list)
    del images, gt_images

Epoch 1/5
311/312 [============================>.] - ETA: 1s - loss: 49.0569
Epoch 00001: loss improved from inf to 49.07083, saving model to drive/My Drive/Machine_Learning-prj/colorization/colorization_12_1.h5
312/312 [==============================] - 582s 2s/step - loss: 49.0708 - val_loss: 54.1165
Epoch 2/5
311/312 [============================>.] - ETA: 1s - loss: 46.4252
Epoch 00002: loss improved from 49.07083 to 46.43437, saving model to drive/My Drive/Machine_Learning-prj/colorization/colorization_12_1.h5
312/312 [==============================] - 560s 2s/step - loss: 46.4344 - val_loss: 54.2824
Epoch 3/5
311/312 [============================>.] - ETA: 1s - loss: 44.5869
Epoch 00003: loss improved from 46.43437 to 44.58549, saving model to drive/My Drive/Machine_Learning-prj/colorization/colorization_12_1.h5
312/312 [==============================] - 560s 2s/step - loss: 44.5855 - val_loss: 57.5667
Epoch 4/5
311/312 [============================>.] - ETA: 1s - loss: 43.2464
E

KeyboardInterrupt: ignored

In [0]:
model_json = model.to_json()
with open("drive/My Drive/Machine_Learning-prj/colorization/model_15_1.json", "w") as json_file:
    json_file.write(model_json)